In [1]:
#importing libraries
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words=list(stopwords.words("english"))

import os
from zipfile import ZipFile
from google.colab import files
uploads=files.upload()                           # preprocessed zipped collection of document files is expected to be uploaded

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Saving CSE508_Winter2023_Dataset3_preprocessed.zip to CSE508_Winter2023_Dataset3_preprocessed.zip


In [2]:
class Inverted_Index:
  def __init__(self):
    self.index_dict = dict()                      # dictionary for storing unigram inverted index
    self.token_list = list()                      # list to store all of the words present in the documents
    self.doc_count = 0                            # variable to store the total count of documents
    self.word_freq = dict()                       # dictionary for storing the count of indices stored for each word in the unigram inverted index
    self.all_doc_ids = []                         # list to store all document id numbers
    self.doc_id_map = dict()                      # dictionary that maps document_id to document_name


  def make_tok_list(self):
    os.mkdir('temp_dir')
    with ZipFile("/content/CSE508_Winter2023_Dataset3_preprocessed.zip", 'r') as zObject:    
      zObject.extractall(path="temp_dir")                                        # Extracting all the files from the zip into a specific location
    dir_path='/content/temp_dir/content/temp_dir/CSE508_Winter2023_Dataset'
    files_list=os.listdir(dir_path)                                              # getting the files in a list          
    i = 0
    for fl in files_list:
      self.doc_id_map[i] = fl
      self.all_doc_ids.append(i)                        # adding the current file's document_id to the list all_doc_ids
      i += 1
      self.doc_count += 1
      fl_path=os.path.join(dir_path,fl)
      fobj=open(fl_path,'r')
      text=fobj.read()
      word_toks = text.split(",")                       # the words in the file are seperated by comma, to extract them spliting wrt ","
      self.token_list.append(word_toks)                 # then adding those words in the token_list, which is collection of all present tokens
      fobj.close()


  def make_unigram(self):
    unique_token_list=set()                                       # stores unique tokens present in token_list
    self.make_tok_list()
    for toks in self.token_list:
      unique_token_list = unique_token_list.union(set(toks))      # getting the unique token from tokens present in token_list using set union
    unique_token_list = list(unique_token_list)                   
    for tok in unique_token_list:
      self.index_dict[tok] = []                                   # initializing each inverted index as empty list
      self.word_freq[tok] = 0                                     # initializing the length of each inverted index as zero
    for doc in range(self.doc_count):                             # generating the unigram inverted index
      for tok in self.token_list[doc]:
        if doc not in self.index_dict[tok]:
          self.index_dict[tok].append(doc)
          self.word_freq[tok] = self.word_freq[tok] + 1
    for tok in unique_token_list:
      self.index_dict[tok].sort()                                 # sorting the document_ids present in each inverted index in increasing order


  def t1_AND_t2(self,doc_list1,doc_list2,l1,l2):          # takes the inverted index list of two operand tokens and their lengths as input
    if l1 == 0 and l2 ==0:                                # if none of the operands present in the index_dict the return empty list
      return [],0
    i, j, comparisons = 0, 0, 0                           # the variable comparisons stores the number of comparisons made for this task
    docs = []
    while i<l1 and j<l2:                                      
      comparisons += 1
      if doc_list1[i] == doc_list2[j]:                    # add the document_ids present in both doc_list1 and doc_list2 in a new list 'docs'
        docs.append(doc_list1[i])
        i += 1
        j += 1
      elif doc_list1[i] < doc_list2[j]:
        i += 1
      else:
        j += 1
    return docs,comparisons                               # return the desired list 'docs', and comparisons_count


  def t1_OR_t2(self,doc_list1,doc_list2,l1,l2):           # takes the inverted index list of two operand tokens and their lengths as input
    if l1!=0 and l2!=0:                                   # condition to check if both of the tokens are present in the inverted index
      i, j, comparisons = 0, 0, 0                         # the variable comparisons stores the number of comparisons made for this task
      docs = []
      while i<l1 and j<l2:                                # merging doc_list1 and doc_list2 into resultant list 'docs' 
        comparisons+=1                                    # it is considered that doc_list1 and doc_list2 may have many common elements
        if doc_list1[i]<doc_list2[j]:
          docs.append(doc_list1[i])
          i=i+1
        elif doc_list1[i]>doc_list2[j]:              
          docs.append(doc_list2[j])
          j=j+1
        else:
          docs.append(doc_list1[i])
          i=i+1
          j=j+1
      docs = docs + doc_list1[i:] + doc_list2[j:]        
      return docs,comparisons                             # return the desired list 'docs', and comparisons_count
    elif l1!=0:                                           # condition to check if only the first token is present in the inverted index
      return doc_list1,0                                  # simply return the inverted index for first token
    elif l2!=0:                                           # condition to check if only the second token is present in the inverted index
      return doc_list2,0                                  # simply return the inverted index for second token
    else:                                                 # if none of the tokens are present in the dictionary the return empty list
      return [],0


  def t1_AND_NOT_t2(self,doc_list1,doc_list2,l1,l2):      # takes the inverted index list of two operand tokens and their lengths as input
    if l1 == 0:                                           # condition to check if the first token is not present in the index_dict
      return [],0                                         # if so then just return empty list
    docs = []
    i, j, comparisons = 0, 0, 0                           # the variable comparisons stores the number of comparisons made for this task
    while i<l1 and j<l2:                                  # run the loop untill at least one of the index-list donesn't exhaust
      comparisons += 1
      if doc_list1[i] == doc_list2[j]:
        i += 1
        j += 1
      elif doc_list1[i] < doc_list2[j]:
        docs.append(doc_list1[i])                         # adding the doc-id's which are in doc_list1 but not in doc_list2 in a new list 'docs'
        i += 1
      else:
        j += 1
    docs = docs + doc_list1[i:]
    return docs,comparisons                              # return the desired list 'docs', and comparisons_count


  def t1_OR_NOT_t2(self,doc_list1,doc_list2,l1,l2):       # takes the inverted index list of two operand tokens and their lengths as input
    if l2 == 0:
      return self.all_doc_ids,0
    docs = self.all_doc_ids.copy()                        # copy all possible document-ids in the result-list 'docs'
    comparisons = 0                                       # the variable comparisons stores the number of comparisons made for this task
    for id in range(doc_list2[0],doc_list2[-1]+1):
      comparisons += 1
      if id in doc_list2:
        docs.remove(id)                                   # if a doc-id is present in doc_list2 then remove it from the result-list 'docs'
    return docs,comparisons                      # return the document_ids present in either doc_list1 or not in doc_list2, and comparisons_count


  def preprocess_query(self,query):                              # function for preprocessing the input query string                   
    query = query.lower()                                        # converting the texts of query string into lowercase
    words = re.sub('[^\w\s]',' ',query)                          # removing non-alphanumeric and non-space characters from the query string
    tokens = nltk.word_tokenize(words)                           # tokenizing the modified query string
    new_tokens = [t for t in tokens if t not in stop_words]      # removing stopwords from the tokens obtained in last step
    words_list = [t for t in new_tokens if t!=' ']               # removing all blank-space tokens
    return words_list                                            # returning a list of filtered tokens


  def query_processing(self,query_sent,operations):
    tokens = self.preprocess_query(query_sent)                   # a list of tokens will be returned here after preprocessing the whole string query
    ops=operations.split(",")                                    # the comma seperated boolean operators received as i/p are stored in list 'ops'
    processed_query = ""                                  # will store a string containing all operands and operators in b/w them in proper sequence
    for i in range(len(tokens)):
      processed_query = processed_query + tokens[i] + " "
      if i != len(tokens)-1:
        processed_query = processed_query + ops[i] + " "
    pos=0
    func_dict={"AND":self.t1_AND_t2,"OR":self.t1_OR_t2,"OR NOT":self.t1_OR_NOT_t2,"AND NOT":self.t1_AND_NOT_t2}            # dictionary containing 
    docs = []                                                            # boolean operators as keys and their respective function names as values
    comparisons, i, j = 0,0,0 
    while i < len(tokens):
      doc_list1, doc_list2 = [], []
      l1, l2 = 0, 0
      if pos==0:                                                         # the first boolean operator will be applied on first two words
        if tokens[0] in self.index_dict:                                 # from the preprocessed i/p sequence 'tokens'
          doc_list1 = self.index_dict[tokens[0]]
          l1 = self.word_freq[tokens[0]]
        if tokens[1] in self.index_dict:
          doc_list2 = self.index_dict[tokens[1]]
          l2 = self.word_freq[tokens[1]]
        docs, count = func_dict[ops[j]](doc_list1,doc_list2,l1,l2)
        pos += 1
        i += 2
      else:                                         # from second boolean operator and onwards 
        if tokens[i] in self.index_dict:            # the 1st operand will be the resultant list of doc-ids obtained from previous boolean operation
          doc_list2 = self.index_dict[tokens[i]]    # and the 2nd operand will be next available word from the preprocessed i/p sequence 'tokens'           
          l2 = self.word_freq[tokens[i]]
        docs, count = func_dict[ops[j]](docs,doc_list2,len(docs),l2)
        i += 1
      j += 1
      comparisons += count                          # comparisons required for performing each operators will be added to get total
                                                    # count of comparisons required for the entire complex query
    return processed_query,docs,comparisons         # returning the desired outputs   


  def input_output(self):                                          # funtion to take the inputs in given format, call the query_processing method 
    n = int(input("Enter number of queries to execute : "))        # and then print the output in desired format
    query = [None for i in range(n)]
    op = [None for i in range(n)]
    for i in range(n):
      query[i] = input("Enter Input sequence : ")
      op[i] = input("Enter Operations separated by comma : ")
    for i in range(n):
      processed_query, doc_ids, comparisons = self.query_processing(query[i],op[i])
      doc_count = len(doc_ids)
      doc_names = [self.doc_id_map[id] for id in doc_ids]
      doc_names.sort()
      names = ""
      for j in range(doc_count):
        if j==doc_count-1:
          names = names + doc_names[j] + ".txt"
        else:
          names = names + doc_names[j] + ".txt" + ", "
      print("\n\nQuery {}: {}".format(i+1,processed_query))
      print("Number of documents retrieved for query {}: {}".format(i+1,doc_count))
      print("Names of the documents retrieved for query {}: {}".format(i+1,names))
      print("Number of comparisons required for query {}: {}".format(i+1,comparisons))
      


In [4]:
obj = Inverted_Index()                  # create object of the Inverted_Index class
obj.make_unigram()                      # generate the unigram inverted index for this instance obj

In [5]:
import pickle as pkl                                
f1=open("Unigram_inverted_index.pkl",'wb')
pkl.dump(obj,f1)                                 # store this class object obj as pkl file for future use
f1.close()

In [6]:
files.download('Unigram_inverted_index.pkl')     # downloading the pl=kl file for storing locally

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [3]:
import pickle as pkl       
uploads=files.upload()                            
f2=open("Unigram_inverted_index.pkl",'rb')
uii=pkl.load(f2)                                  # load the pkl file stored earlier
f2.close() 

Saving Unigram_inverted_index.pkl to Unigram_inverted_index.pkl


In [6]:
uii.input_output()          # call the input_output method using pkl object

Enter number of queries to execute : 2
Enter Input sequence : Details of Design, and Review.
Enter Operations separated by comma : AND,OR NOT
Enter Input sequence : Body in Shapes, is well !!!!
Enter Operations separated by comma : AND,AND NOT


Query 1: details AND design OR NOT review 
Number of documents retrieved for query 1: 1377
Names of the documents retrieved for query 1: cranfield0001.txt, cranfield0002.txt, cranfield0003.txt, cranfield0004.txt, cranfield0005.txt, cranfield0006.txt, cranfield0007.txt, cranfield0008.txt, cranfield0009.txt, cranfield0010.txt, cranfield0011.txt, cranfield0012.txt, cranfield0013.txt, cranfield0014.txt, cranfield0015.txt, cranfield0016.txt, cranfield0017.txt, cranfield0018.txt, cranfield0019.txt, cranfield0020.txt, cranfield0021.txt, cranfield0022.txt, cranfield0023.txt, cranfield0024.txt, cranfield0025.txt, cranfield0026.txt, cranfield0027.txt, cranfield0028.txt, cranfield0029.txt, cranfield0030.txt, cranfield0031.txt, cranfield0032.txt, cranfield

In [7]:
uii.input_output()          # call the input_output method using pkl object

Enter number of queries to execute : 1
Enter Input sequence : Profiles for Match !!
Enter Operations separated by comma : OR


Query 1: profiles OR match 
Number of documents retrieved for query 1: 68
Names of the documents retrieved for query 1: cranfield0049.txt, cranfield0054.txt, cranfield0061.txt, cranfield0071.txt, cranfield0074.txt, cranfield0076.txt, cranfield0082.txt, cranfield0088.txt, cranfield0094.txt, cranfield0101.txt, cranfield0138.txt, cranfield0165.txt, cranfield0168.txt, cranfield0171.txt, cranfield0177.txt, cranfield0178.txt, cranfield0189.txt, cranfield0206.txt, cranfield0211.txt, cranfield0218.txt, cranfield0219.txt, cranfield0233.txt, cranfield0240.txt, cranfield0261.txt, cranfield0327.txt, cranfield0328.txt, cranfield0344.txt, cranfield0351.txt, cranfield0364.txt, cranfield0365.txt, cranfield0366.txt, cranfield0370.txt, cranfield0387.txt, cranfield0397.txt, cranfield0417.txt, cranfield0435.txt, cranfield0440.txt, cranfield0474.txt, cranfield0491.txt, cranfield049